In [85]:
import duckdb

In [86]:
con = duckdb.connect()
users_df = con.sql("""select * from read_csv_auto('data/users.csv')""").to_df()
videos_df = con.sql("""select * from read_csv_auto('data/videos.csv')""").to_df()
devices_df = con.sql("""select * from read_csv_auto('data/devices.csv')""").to_df()
events_df = con.sql("""select * from read_json_auto('data/events.jsonl')""").to_df()
con.register('users', users_df)
con.register('videos', videos_df)
con.register('devices', devices_df)
con.register('events', events_df)

In [87]:
# some preparations

first_session_df = con.execute(f"""

select user_id, session_id, sum(value) total_watch_time from read_json_auto('/Users/fvyshkov/train/data/events.jsonl')
where event_name = 'watch_time'
and session_id in (
SELECT session_id FROM read_json_auto('/Users/fvyshkov/train/data/events.jsonl')
where event_name = 'first_login'
)
group by user_id, session_id
""").fetch_df()

con.register('first_sessions', first_session_df)

In [88]:
# Q1: What % of new users reach at least 30 seconds of watch_time in their first session?

ratio_df = con.execute("""

select sum(total_watch_time)/(select sum(total_watch_time) from first_sessions) as percent
from first_sessions where total_watch_time >= 30

""").fetch_df()

print("Q1: What % of new users reach at least 30 seconds of watch_time in their first session?")
PERSENT_MORE_30_FIRST = ratio_df['percent'][0]
print(f"{round(PERSENT_MORE_30_FIRST*100)}%")

Q1: What % of new users reach at least 30 seconds of watch_time in their first session?
94%


In [92]:
# Q2: “Which video genres drive the highest 2nd-session retention within 3 days?”
retention_3d_df = con.execute(f"""

with first_session_start as (
  select e.user_id, fs.session_id, min(cast(e.timestamp as timestamp)) as first_start_ts
  from events e
  join first_sessions fs on fs.session_id = e.session_id
  where e.event_name = 'session_start'
  group by e.user_id, fs.session_id
),
returned_within_3d as (
  select fss.user_id,
         case when exists (
           select 1 from events e2
           where e2.event_name = 'session_start'
             and e2.user_id = fss.user_id
             and e2.session_id <> fss.session_id
             and cast(e2.timestamp as timestamp) > fss.first_start_ts
             and cast(e2.timestamp as timestamp) <= fss.first_start_ts + interval 3 day
         ) then 1 else 0 end as returned_3d
  from first_session_start fss
),
first_session_watch_by_genre as (
  select e.user_id, e.session_id, v.genre, sum(e.value) as watch_time_genre
  from events e
  join first_sessions fs on fs.session_id = e.session_id
  join videos v on v.video_id = e.video_id
  where e.event_name = 'watch_time'
  group by e.user_id, e.session_id, v.genre
),
eligible as (
  -- threshold for gwenre - 30 sec
  select * from first_session_watch_by_genre where watch_time_genre >= 30
)
select
  genre,
  sum(watch_time_genre) as first_session_watch_time,
  count(distinct case when r.returned_3d = 1 then e.user_id end) as returned_users_3d,
  count(distinct e.user_id) as total_users,
  count(distinct case when r.returned_3d = 1 then e.user_id end)::decimal / nullif(count(distinct e.user_id), 0) as retention_3d
from eligible e,  returned_within_3d r
 where r.user_id = e.user_id
group by genre
order by retention_3d desc

""").fetch_df()

print("Which video genres drive the highest 2nd-session retention within 3 days? Starting from top genres")
retention_3d_df['genre']

Which video genres drive the highest 2nd-session retention within 3 days? Starting from top genres


0         horror
1         sci-fi
2         action
3          drama
4        romance
5         comedy
6    documentary
7        fantasy
Name: genre, dtype: object

In [91]:
# Q3: “Is there a particular device_os or app_version where drop-off is abnormally high?”

drop_off_df = con.execute(f"""

with session_watch as (
    select
        e.session_id,
        e.device_os,
        e.app_version,
        sum(case when e.event_name = 'watch_time' then coalesce(e.value, 0) else 0 end) as total_watch_time
    from events e,  first_sessions fs
     where fs.session_id = e.session_id
    group by e.session_id, e.device_os, e.app_version
),
session_flags as (
    select
        session_id,
        device_os,
        app_version,
        case when total_watch_time < 10 then 1 else 0 end as is_dropoff
    from session_watch
)
select
    device_os,
    app_version,
    count(*) as total_first_sessions,
    sum(is_dropoff) as dropoff_sessions,
    round(100.0 * sum(is_dropoff) / count(*), 2) as dropoff_rate_pct
from session_flags
group by device_os, app_version
order by dropoff_rate_pct desc



""").fetch_df()

print("Q3: Is there a particular device_os or app_version where drop-off is abnormally high?")
drop_off_df[['device_os', 'app_version', 'dropoff_rate_pct']]

Q3: Is there a particular device_os or app_version where drop-off is abnormally high?


,device_os,app_version,dropoff_rate_pct
0,macOS,1.0.0,16.67
1,iOS,2.0.0,15.38
2,macOS,1.2.0,14.29
3,Android,2.1.0,13.33
4,iOS,1.2.0,13.33
5,Android,1.1.0,12.50
6,macOS,1.1.0,12.50
7,Android,2.0.0,12.50
8,Android,1.2.0,12.50
9,Windows,1.1.0,11.11
